In [1]:
import warnings
warnings.filterwarnings('ignore') #filtering warnings
import numpy as np # linear algebra
import pandas as pd # data processing, read csv file
import pandas as pd

from sklearn.model_selection import cross_val_score, StratifiedKFold
from scipy.sparse import hstack
from matplotlib import pyplot as plt
import unidecode
import re


# Input data files are available in the "../input/" directory.

import os
for dirname, _, filenames in os.walk('E:/Data science/AV/bbc txt classification/bbc/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('E:/Data science/AV/bbc txt classification/bbc/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

E:/Data science/AV/bbc txt classification/bbc/README.TXT
E:/Data science/AV/bbc txt classification/bbc/business\002.txt
E:/Data science/AV/bbc txt classification/bbc/business\003.txt
E:/Data science/AV/bbc txt classification/bbc/business\004.txt
E:/Data science/AV/bbc txt classification/bbc/business\005.txt
E:/Data science/AV/bbc txt classification/bbc/business\006.txt
E:/Data science/AV/bbc txt classification/bbc/business\007.txt
E:/Data science/AV/bbc txt classification/bbc/business\008.txt
E:/Data science/AV/bbc txt classification/bbc/business\009.txt
E:/Data science/AV/bbc txt classification/bbc/business\011.txt
E:/Data science/AV/bbc txt classification/bbc/business\012.txt
E:/Data science/AV/bbc txt classification/bbc/business\013.txt
E:/Data science/AV/bbc txt classification/bbc/business\014.txt
E:/Data science/AV/bbc txt classification/bbc/business\015.txt
E:/Data science/AV/bbc txt classification/bbc/business\016.txt
E:/Data science/AV/bbc txt classification/bbc/business\017.tx

In [2]:
# Step 1 - Get the file details
directory = []
file = []
title = []
text = []
label = []
datapath = 'E:/Data science/AV/bbc txt classification/bbc/' 
for dirname, _ , filenames in os.walk(datapath):
    #print('Directory: ', dirname)
    #print('Subdir: ', dirname.split('/')[-1])
    # remove the Readme.txt file
    # will not find file in the second iteration so we skip the error
    try:
        filenames.remove('README.TXT')
    except:
        pass
    for filename in filenames:
        directory.append(dirname)
        file.append(filename)
        label.append(dirname.split('/')[-1])
        #print(filename)
        fullpathfile = os.path.join(dirname,filename)
        with open(fullpathfile, 'r', encoding="utf8", errors='ignore') as infile:
            intext = ''
            firstline = True
            for line in infile:
                if firstline:
                    title.append(line.replace('\n',''))
                    firstline = False
                else:
                    intext = intext + ' ' + line.replace('\n','')
            text.append(intext)

Construct a dataframe each row contains each text file from all categories. First line in the text file will be under column heading 'title', rest of the content will be under 'text' column heading and category name will be under 'label'.                                            

In [3]:
fulldf = pd.DataFrame(list(zip(directory, file, title, text, label)), 
               columns =['directory', 'file', 'title', 'text', 'label'])

df = fulldf.filter(['title','text','label'], axis=1)

print("FullDf : ", fulldf.shape)
print("DF : ", df.shape)

FullDf :  (2215, 5)
DF :  (2215, 3)


In [4]:
fulldf.head()

,directory,file,title,text,label
0,E:/Data science/AV/bbc txt classification/bbc/...,002.txt,Dollar gains on Greenspan speech,The dollar has hit its highest level against...,business
1,E:/Data science/AV/bbc txt classification/bbc/...,003.txt,Yukos unit buyer faces loan claim,The owners of embattled Russian oil giant Yu...,business
2,E:/Data science/AV/bbc txt classification/bbc/...,004.txt,High fuel prices hit BA's profits,British Airways has blamed high fuel prices ...,business
3,E:/Data science/AV/bbc txt classification/bbc/...,005.txt,Pernod takeover talk lifts Domecq,Shares in UK drinks and food firm Allied Dom...,business
4,E:/Data science/AV/bbc txt classification/bbc/...,006.txt,Japan narrowly escapes recession,Japan's economy teetered on the brink of a t...,business


In [5]:
df.head()

,title,text,label
0,Dollar gains on Greenspan speech,The dollar has hit its highest level against...,business
1,Yukos unit buyer faces loan claim,The owners of embattled Russian oil giant Yu...,business
2,High fuel prices hit BA's profits,British Airways has blamed high fuel prices ...,business
3,Pernod takeover talk lifts Domecq,Shares in UK drinks and food firm Allied Dom...,business
4,Japan narrowly escapes recession,Japan's economy teetered on the brink of a t...,business


In [6]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2)
print("Train DF: ",train.shape)
print("Test DF: ",test.shape)

Train DF:  (1772, 3)
Test DF:  (443, 3)


## Preprocessing 

In [7]:
def remove_accented_chars(text):
    """remove accented characters from text, e.g. café"""
    text = unidecode.unidecode(text)
    return text

In [8]:
def lower_(text):
    return text.lower()

In [9]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
  
def stop_words_removal(sentence):
  
    stop_words = set(stopwords.words('english')) 
    word_tokens = word_tokenize(sentence)
  
    filtered_sentence = [w for w in word_tokens if not w in stop_words] 
    return (' '.join(filtered_sentence))

In [10]:
def remove_special_characters(text, remove_digits=True):
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text

In [11]:
train['cons']=train['title']+train['text'] 
test['cons']=test['title']+test['text'] 

In [12]:
train['cons']=train['cons'].apply(remove_accented_chars)
test['cons']=test['cons'].apply(remove_accented_chars)

train['cons']=train['cons'].apply(lower_)
test['cons']=test['cons'].apply(lower_)

train['cons']=train['cons'].apply(stop_words_removal)
test['cons']=test['cons'].apply(stop_words_removal)

train['cons']=train['cons'].apply(remove_special_characters)
test['cons']=test['cons'].apply(remove_special_characters)

In [13]:
train.head()

,title,text,label,cons
1499,Juninho demand for O'Neill talks,Juninho's agent has confirmed that the playe...,sport,juninho demand oneill talks juninho s agent co...
1678,Hong Kong in 2011 World Cup bid,Hong Kong is hoping to join Japan as co-host...,sport,hong kong world cup bid hong kong hoping join...
1817,China net cafe culture crackdown,"Chinese authorities closed 12,575 net cafes ...",tech,china net cafe culture crackdown chinese autho...
973,Job cuts 'false economy' - TUC,"Plans to shed 71,000 civil service jobs will...",politics,job cuts false economy tuc plans shed civil...
1697,Bath faced with Tindall ultimatum,Mike Tindall's agent has warned Bath they ha...,sport,bath faced tindall ultimatum mike tindall s ag...


In [14]:
train_data=train.drop(['cons'],axis=1)

In [15]:
train=train.drop(['title','text'],axis=1)

In [16]:
test_data=test.drop(['cons'],axis=1)

In [17]:
test=test.drop(['title','text'],axis=1)

In [18]:
x_train=train['cons']
y_train=train['label']
x_test=test['cons']
y_test=test['label']

In [19]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(x_train)

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(x_train)

#### Model building
##### Multinomial Naive Bayes

In [21]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [22]:
X_test_tfidf = vectorizer.transform(x_test)
print ('Dimension of TF-IDF vector :' , X_test_tfidf.shape)

predicted = clf.predict(X_test_tfidf)

Dimension of TF-IDF vector : (443, 27307)


In [23]:
predicted = clf.predict(X_test_tfidf)

In [24]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print('confusion matrix:','\n',confusion_matrix(y_test, predicted),'\n')
print('classification_report:','\n',classification_report(y_test, predicted))
print('Accuracy:',accuracy_score(y_test, predicted))

confusion matrix: 
 [[102   0   3   0   1]
 [  2  75   1   0   1]
 [  1   0  78   0   1]
 [  0   0   0 105   0]
 [  0   0   0   0  73]] 

classification_report: 
                precision    recall  f1-score   support

     business       0.97      0.96      0.97       106
entertainment       1.00      0.95      0.97        79
     politics       0.95      0.97      0.96        80
        sport       1.00      1.00      1.00       105
         tech       0.96      1.00      0.98        73

     accuracy                           0.98       443
    macro avg       0.98      0.98      0.98       443
 weighted avg       0.98      0.98      0.98       443

Accuracy: 0.9774266365688488


Do the same process (converting into dataframe and preprocessing) for test data and predict the label using our built model.

In [25]:
import os
for dirname, _, filenames in os.walk('E:/Data science/AV/bbc txt classification/test/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

E:/Data science/AV/bbc txt classification/test/001.txt
E:/Data science/AV/bbc txt classification/test/002.txt
E:/Data science/AV/bbc txt classification/test/003.txt
E:/Data science/AV/bbc txt classification/test/004.txt
E:/Data science/AV/bbc txt classification/test/005.txt
E:/Data science/AV/bbc txt classification/test/006.txt
E:/Data science/AV/bbc txt classification/test/007.txt
E:/Data science/AV/bbc txt classification/test/008.txt
E:/Data science/AV/bbc txt classification/test/009.txt
E:/Data science/AV/bbc txt classification/test/010.txt


In [26]:
# Step 1 - Get the file details
directory = []
file = []
title = []
text = []
label = []
datapath = 'E:/Data science/AV/bbc txt classification/test/' 
for dirname, _ , filenames in os.walk(datapath):
    #print('Directory: ', dirname)
    #print('Subdir: ', dirname.split('/')[-1])
    # remove the Readme.txt file
    # will not find file in the second iteration so we skip the error
    try:
        filenames.remove('README.TXT')
    except:
        pass
    for filename in filenames:
        directory.append(dirname)
        file.append(filename)
        label.append(dirname.split('/')[-1])
        #print(filename)
        fullpathfile = os.path.join(dirname,filename)
        with open(fullpathfile, 'r', encoding="utf8", errors='ignore') as infile:
            intext = ''
            firstline = True
            for line in infile:
                if firstline:
                    title.append(line.replace('\n',''))
                    firstline = False
                else:
                    intext = intext + ' ' + line.replace('\n','')
            text.append(intext)

In [27]:
sample = pd.DataFrame(list(zip(directory, file, title, text, label)), 
               columns =['directory', 'file', 'title', 'text', 'label'])

sample_df = sample.filter(['title','text'], axis=1)

sample_df

,title,text
0,Ad sales boost Time Warner profit,Quarterly profits at US media giant TimeWarn...
1,Jarre joins fairytale celebration,French musician Jean-Michel Jarre is to perf...
2,Hewitt decries 'career sexism',Plans to extend paid maternity leave beyond ...
3,IAAF launches fight against drugs,The IAAF - athletics' world governing body -...
4,Technology gets the creative bug,The hi-tech and the arts worlds have for som...
5,Wi-fi web reaches farmers in Peru,"A network of community computer centres, lin..."
6,O'Sullivan commits to Dublin race,Sonia O'Sullivan will seek to regain her tit...
7,Women MPs reveal sexist taunts,"Women MPs endure ""shocking"" levels of sexist..."
8,Da Vinci Code is 'lousy history',The plot of an international bestseller that...
9,Court rejects $280bn tobacco case,A US government claim accusing the country's...


In [28]:
sample_data=sample_df.copy()

In [29]:
sample_df['cons']=sample_df['title']+sample_df['text'] 

In [30]:
sample_df['cons']=sample_df['cons'].apply(lower_)
sample_df['cons']=sample_df['cons'].apply(remove_accented_chars)

sample_df['cons']=sample_df['cons'].apply(stop_words_removal)


sample_df['cons']=sample_df['cons'].apply(remove_special_characters)


In [31]:
sample_df=sample_df.drop(['title','text'],axis=1)

In [32]:
sample_df.head()

,cons
0,ad sales boost time warner profit quarterly pr...
1,jarre joins fairytale celebration french music...
2,hewitt decries career sexism plans extend pai...
3,iaaf launches fight drugs iaaf athletics wor...
4,technology gets creative bug hitech arts world...


In [33]:
sample_test=sample_df['cons']

In [34]:
X_sample_test_tfidf = vectorizer.transform(sample_test)
print ('Dimension of TF-IDF vector :' , X_sample_test_tfidf.shape)

pred = clf.predict(X_sample_test_tfidf)

Dimension of TF-IDF vector : (10, 27307)


In [35]:
import pandas as pd
sample_prediction=pd.DataFrame({'title':sample_data['title'],'text':sample_data['text'],'prediction':pred})

In [36]:
sample_prediction

,title,text,prediction
0,Ad sales boost Time Warner profit,Quarterly profits at US media giant TimeWarn...,business
1,Jarre joins fairytale celebration,French musician Jean-Michel Jarre is to perf...,entertainment
2,Hewitt decries 'career sexism',Plans to extend paid maternity leave beyond ...,politics
3,IAAF launches fight against drugs,The IAAF - athletics' world governing body -...,sport
4,Technology gets the creative bug,The hi-tech and the arts worlds have for som...,tech
5,Wi-fi web reaches farmers in Peru,"A network of community computer centres, lin...",tech
6,O'Sullivan commits to Dublin race,Sonia O'Sullivan will seek to regain her tit...,sport
7,Women MPs reveal sexist taunts,"Women MPs endure ""shocking"" levels of sexist...",politics
8,Da Vinci Code is 'lousy history',The plot of an international bestseller that...,entertainment
9,Court rejects $280bn tobacco case,A US government claim accusing the country's...,business
